In [ ]:
from google.colab import files
import os
import zipfile
uploaded = files.upload()


Saving Tumour-20251109T042749Z-1-001.zip to Tumour-20251109T042749Z-1-001.zip


In [ ]:
#uploaded a zip, extract it
for fn in uploaded.keys():
    if fn.lower().endswith('.zip'):
        print("Extracting", fn)
        with zipfile.ZipFile(fn, 'r') as z:
            z.extractall('/content/data')
        print("Extracted to /content/data")
    else:
        # If images were uploaded directly, move them into data folder
        os.makedirs('/content/data', exist_ok=True)
        os.rename(fn, os.path.join('/content/data', fn))
print("Done. Check /content/data")

Extracting Tumour-20251109T042749Z-1-001.zip
Extracted to /content/data
Done. Check /content/data


In [ ]:
# Cell 3 — inspect the extracted folder structure and class counts
import os

data_root = '/content/data'  # change if different
for root, dirs, files in os.walk(data_root):
    level = root.replace(data_root, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/ ({len(files)} files)")
    if level == 2:
        # only print a few directories to avoid long outputs
        pass

# Show class counts if dataset arranged as train/<class>/
train_dir = os.path.join(data_root, 'train')
if os.path.exists(train_dir):
    for cls in sorted(os.listdir(train_dir)):
        cls_path = os.path.join(train_dir, cls)
        if os.path.isdir(cls_path):
            print(f"Class '{cls}': {len(os.listdir(cls_path))} images")
else:
    print("No 'train' folder found at", train_dir, "- you may have a different layout.")


data/ (0 files)
  Tumour/ (1 files)
    valid/ (1 files)
      meningioma/ (124 files)
      no_tumor/ (99 files)
      glioma/ (161 files)
      pituitary/ (118 files)
    test/ (1 files)
      meningioma/ (63 files)
      no_tumor/ (49 files)
      glioma/ (80 files)
      pituitary/ (54 files)
    train/ (1 files)
      meningioma/ (358 files)
      no_tumor/ (335 files)
      glioma/ (564 files)
      pituitary/ (438 files)
No 'train' folder found at /content/data/train - you may have a different layout.


In [ ]:
# Auto-detect dataset root and print class counts (run this cell)
import os, itertools, glob

SEARCH_BASE = '/content'   # change if your files live somewhere else

expected_sets = {'train','valid','validation','test'}
min_class_count = 2  # minimum number of class subfolders to consider a match

def is_train_valid_test_dir(path):
    # returns True if directory has train & (valid or validation) or test subfolders
    items = set(os.listdir(path))
    return ('train' in items) and (('valid' in items or 'validation' in items) or ('test' in items))

def has_many_class_dirs(path):
    # returns True if path contains >= min_class_count subdirectories (class-like)
    try:
        subs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path,d))]
        return len(subs) >= min_class_count
    except Exception:
        return False

candidates = []
# walk shallowly to avoid long runs; depth up to 3
for root, dirs, files in os.walk(SEARCH_BASE):
    depth = root.replace(SEARCH_BASE, '').count(os.sep)
    if depth > 3:
        continue
    # case A: folder containing train/valid/test
    if is_train_valid_test_dir(root):
        candidates.append(('set-split', root))
        continue
    # case B: folder that appears to contain class folders directly
    if has_many_class_dirs(root):
        # check if subfolders look like classes (contain images)
        subdirs = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root,d))]
        # count how many subdirs contain image files
        class_like = 0
        for s in subdirs:
            cnt = len(glob.glob(os.path.join(root, s, '*.*')))
            if cnt>0:
                class_like += 1
        if class_like >= min_class_count:
            candidates.append(('class-folders', root))

# Remove duplicates, sort by depth (prefer deeper matches)
unique = []
seen = set()
for kind, p in sorted(candidates, key=lambda x: x[1].count(os.sep), reverse=True):
    if p not in seen:
        unique.append((kind,p)); seen.add(p)

if not unique:
    print("⚠️ No likely dataset root found under", SEARCH_BASE)
    print("Run `!ls -la /content` to inspect uploaded files or re-run upload/extraction.")
else:
    print(f"✅ Found {len(unique)} candidate(s). Showing details...\n")
    for i, (kind, path) in enumerate(unique, 1):
        print(f"--- Candidate {i}: ({kind}) {path} ---")
        # Print immediate children summary
        for child in sorted(os.listdir(path)):
            child_path = os.path.join(path, child)
            if os.path.isdir(child_path):
                cnt = len(glob.glob(os.path.join(child_path, '*.*')))
                print(f"  📁 {child}/  — {cnt} files inside")
        # If this candidate is split into train/valid/test, show counts per class
        if is_train_valid_test_dir(path):
            for setname in ['train','valid','validation','test']:
                set_path = os.path.join(path, setname)
                if os.path.exists(set_path):
                    print(f"\n  -> {setname}/ classes and counts:")
                    for cls in sorted(os.listdir(set_path)):
                        cls_path = os.path.join(set_path, cls)
                        if os.path.isdir(cls_path):
                            print(f"     - {cls}: {len(os.listdir(cls_path))} files")
        else:
            # show class dirs (top-level classes)
            print("\n  -> Top-level class folders (if any):")
            for cls in sorted(os.listdir(path)):
                cls_path = os.path.join(path, cls)
                if os.path.isdir(cls_path):
                    print(f"     - {cls}: {len(glob.glob(os.path.join(cls_path,'*.*')))} files")
        print("\n")

    # choose the best candidate (prefers a set-split match then class-folders)
    chosen = None
    for kind, p in unique:
        if kind == 'set-split':
            chosen = p; break
    if not chosen:
        chosen = unique[0][1]

    print(f"➡️ Selecting dataset root: {chosen}")
    # set data_root variable for the rest of your notebook
    data_root = chosen
    print("Set variable: data_root =", data_root)



✅ Found 5 candidate(s). Showing details...

--- Candidate 1: (class-folders) /content/data/Tumour/valid ---
  📁 glioma/  — 161 files inside
  📁 meningioma/  — 124 files inside
  📁 no_tumor/  — 99 files inside
  📁 pituitary/  — 118 files inside

  -> Top-level class folders (if any):
     - glioma: 161 files
     - meningioma: 124 files
     - no_tumor: 99 files
     - pituitary: 118 files


--- Candidate 2: (class-folders) /content/data/Tumour/test ---
  📁 glioma/  — 80 files inside
  📁 meningioma/  — 63 files inside
  📁 no_tumor/  — 49 files inside
  📁 pituitary/  — 54 files inside

  -> Top-level class folders (if any):
     - glioma: 80 files
     - meningioma: 63 files
     - no_tumor: 49 files
     - pituitary: 54 files


--- Candidate 3: (class-folders) /content/data/Tumour/train ---
  📁 glioma/  — 564 files inside
  📁 meningioma/  — 358 files inside
  📁 no_tumor/  — 335 files inside
  📁 pituitary/  — 438 files inside

  -> Top-level class folders (if any):
     - glioma: 564 fil

In [ ]:
# -----------------------
# Step: create tf.data datasets from detected root
# -----------------------
import os
import tensorflow as tf

# Use the detected root
data_root = '/content/data/Tumour'   # <-- selected by your detector

# Accept either 'valid' or 'validation'
train_dir = os.path.join(data_root, 'train')
val_dir_candidates = [os.path.join(data_root, 'valid'), os.path.join(data_root, 'validation')]
val_dir = next((p for p in val_dir_candidates if os.path.exists(p)), None)
test_dir = os.path.join(data_root, 'test')

print("Using paths:")
print(" train_dir =", train_dir)
print(" val_dir   =", val_dir)
print(" test_dir  =", test_dir)
print()

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123

# Load datasets (will error if paths don't exist)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

if val_dir is not None:
    val_ds = tf.keras.utils.image_dataset_from_directory(
        val_dir,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        seed=SEED
    )
else:
    # fallback: create validation split from train (20%)
    print("No separate valid/validation folder found — creating validation split from train_ds (20%).")
    val_size = int(0.2 * len(train_ds))
    val_ds = train_ds.take(val_size)
    train_ds = train_ds.skip(val_size)

# Load test if present
if os.path.exists(test_dir):
    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False
    )
else:
    test_ds = None
    print("No test folder found at", test_dir)

# Print classes and quick counts
class_names = train_ds.class_names
print("\nDetected classes:", class_names)
print(f"Train batches: {len(train_ds)} | Validation batches: {len(val_ds)} | Test batches: {len(test_ds) if test_ds is not None else 'None'}")

# Performance: cache + prefetch (and normalize to [0,1])
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).prefetch(AUTOTUNE)
val_ds   = val_ds.cache().map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).prefetch(AUTOTUNE)
if test_ds is not None:
    test_ds  = test_ds.cache().map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).prefetch(AUTOTUNE)

print("\nDatasets ready — you can now proceed to build/train your model.")


Using paths:
 train_dir = /content/data/Tumour/train
 val_dir   = /content/data/Tumour/valid
 test_dir  = /content/data/Tumour/test

Found 1695 files belonging to 4 classes.
Found 502 files belonging to 4 classes.
Found 246 files belonging to 4 classes.

Detected classes: ['glioma', 'meningioma', 'no_tumor', 'pituitary']
Train batches: 53 | Validation batches: 16 | Test batches: 8

Datasets ready — you can now proceed to build/train your model.


In [ ]:
# augmentation & normalization layers
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.08),
    layers.RandomZoom(0.08),
    layers.RandomTranslation(0.05, 0.05),
], name="data_augmentation")

preprocess_input = tf.keras.applications.efficientnet.preprocess_input  # for transfer models later

# Apply caching and prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).prefetch(AUTOTUNE)
val_ds = val_ds.cache().map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).prefetch(AUTOTUNE)
if test_ds is not None:
    test_ds = test_ds.cache().map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)).prefetch(AUTOTUNE)


In [ ]:
# custom CNN model
from tensorflow.keras import layers, models, callbacks, optimizers

num_classes = len(class_names)

def get_custom_cnn(input_shape=IMG_SIZE + (3,)):
    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs, name='custom_cnn')
    return model

model_cnn = get_custom_cnn()
model_cnn.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
model_cnn.summary()


Model: "custom_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,172 (434.27 KB)

 Trainable params: 110,724 (432.52 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# train custom CNN (callbacks)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
mc = ModelCheckpoint('custom_cnn_best.h5', monitor='val_loss', save_best_only=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history_cnn = model_cnn.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=[es, mc, rlr]
)


Epoch 1/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5498 - loss: 1.0931

62/62 ━━━━━━━━━━━━━━━━━━━━ 282s 4s/step - accuracy: 0.5537 - loss: 1.0895 - val_accuracy: 1.0000 - val_loss: 0.9310 - learning_rate: 1.0000e-04
Epoch 2/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9981 - loss: 0.4528

62/62 ━━━━━━━━━━━━━━━━━━━━ 307s 4s/step - accuracy: 0.9981 - loss: 0.4519 - val_accuracy: 1.0000 - val_loss: 0.3326 - learning_rate: 1.0000e-04
Epoch 3/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 1.0000 - loss: 0.2632

62/62 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 1.0000 - loss: 0.2628 - val_accuracy: 1.0000 - val_loss: 0.0869 - learning_rate: 1.0000e-04
Epoch 4/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 1.0000 - loss: 0.1550

62/62 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 1.0000 - loss: 0.1548 - val_accuracy: 1.0000 - val_loss: 0.0267 - learning_rate: 1.0000e-04
Epoch 5/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 1.0000 - loss: 0.0990

62/62 ━━━━━━━━━━━━━━━━━━━━ 268s 4s/step - accuracy: 1.0000 - loss: 0.0989 - val_accuracy: 1.0000 - val_loss: 0.0148 - learning_rate: 1.0000e-04
Epoch 6/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 1.0000 - loss: 0.0787

62/62 ━━━━━━━━━━━━━━━━━━━━ 256s 4s/step - accuracy: 1.0000 - loss: 0.0786 - val_accuracy: 1.0000 - val_loss: 0.0128 - learning_rate: 1.0000e-04
Epoch 7/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 255s 4s/step - accuracy: 1.0000 - loss: 0.0561 - val_accuracy: 1.0000 - val_loss: 0.0152 - learning_rate: 1.0000e-04
Epoch 8/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 257s 4s/step - accuracy: 1.0000 - loss: 0.0430 - val_accuracy: 1.0000 - val_loss: 0.0220 - learning_rate: 1.0000e-04
Epoch 9/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 260s 4s/step - accuracy: 1.0000 - loss: 0.0340 - val_accuracy: 1.0000 - val_loss: 0.0211 - learning_rate: 1.0000e-04
Epoch 10/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 256s 4s/step - accuracy: 1.0000 - loss: 0.0288 - val_accuracy: 1.0000 - val_loss: 0.0222 - learning_rate: 5.0000e-05
Epoch 11/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 275s 4s/step - accuracy: 1.0000 - loss: 0.0239 - val_accuracy: 1.0000 - val_loss: 0.0231 - learning_rate: 5.0000e-05
Epoch 12/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 258s 4s/step - accuracy: 1.0000 - loss: 

In [ ]:
# Cell 8 — transfer learning model (EfficientNetB0)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=IMG_SIZE + (3,))
base_model.trainable = False  # freeze base initially

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)  # augmentation in-model
x = layers.Lambda(lambda z: z * 255.0)(x)  # because preprocess expects 0-255 for EfficientNet preprocess
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
tl_model = models.Model(inputs, outputs, name='EfficientNetB0_TL')

tl_model.compile(optimizer=optimizers.Adam(1e-3),
                 loss='sparse_categorical_crossentropy', metrics=['accuracy'])
tl_model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "EfficientNetB0_TL"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,378,535 (16.70 MB)

 Trainable params: 328,964 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# train the transfer model top layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

es_tl = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
mc_tl = ModelCheckpoint('effnet_top_best.h5', monitor='val_loss', save_best_only=True)
rlr_tl = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history_tl = tl_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[es_tl, mc_tl, rlr_tl]
)


Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9202 - loss: 0.1784

62/62 ━━━━━━━━━━━━━━━━━━━━ 208s 3s/step - accuracy: 0.9212 - loss: 0.1762 - val_accuracy: 1.0000 - val_loss: 3.0051e-08 - learning_rate: 0.0010
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - loss: 3.5293e-07

62/62 ━━━━━━━━━━━━━━━━━━━━ 179s 3s/step - accuracy: 1.0000 - loss: 3.5183e-07 - val_accuracy: 1.0000 - val_loss: 2.8809e-08 - learning_rate: 0.0010
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - loss: 2.2721e-07

62/62 ━━━━━━━━━━━━━━━━━━━━ 190s 3s/step - accuracy: 1.0000 - loss: 2.2754e-07 - val_accuracy: 1.0000 - val_loss: 2.8561e-08 - learning_rate: 0.0010
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 177s 3s/step - accuracy: 1.0000 - loss: 2.1873e-07 - val_accuracy: 1.0000 - val_loss: 2.8561e-08 - learning_rate: 0.0010
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 1.0000 - loss: 3.4925e-07 - val_accuracy: 1.0000 - val_loss: 2.8561e-08 - learning_rate: 5.0000e-04
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 177s 3s/step - accuracy: 1.0000 - loss: 2.4244e-07 - val_accuracy: 1.0000 - val_loss: 2.8561e-08 - learning_rate: 5.0000e-04
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 177s 3s/step - accuracy: 1.0000 - loss: 1.8091e-07 - val_accuracy: 1.0000 - val_loss: 2.8561e-08 - learning_rate: 5.0000e-04
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 1.0000 - loss: 1.5810e-07 - val_accuracy: 1.0000 - val_loss: 2.8561e-08 - learning_rate: 2.5000e-04


In [ ]:
# optional: fine-tune top layers of base_model
base_model.trainable = True

# Freeze all layers except last N layers
fine_tune_at = len(base_model.layers) - 40  # unfreeze last 40 layers (adjustable)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

tl_model.compile(optimizer=optimizers.Adam(1e-5),
                 loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_finetune = tl_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[es_tl, mc_tl, rlr_tl]
)


Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 242s 4s/step - accuracy: 0.6627 - loss: 1.0414 - val_accuracy: 1.0000 - val_loss: 3.2288e-04 - learning_rate: 1.0000e-05
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.9230 - loss: 0.3280 - val_accuracy: 1.0000 - val_loss: 0.0052 - learning_rate: 1.0000e-05
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 282s 4s/step - accuracy: 0.9845 - loss: 0.1409 - val_accuracy: 0.9979 - val_loss: 0.0130 - learning_rate: 1.0000e-05
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 205s 3s/step - accuracy: 0.9940 - loss: 0.0724 - val_accuracy: 0.9979 - val_loss: 0.0179 - learning_rate: 1.0000e-05
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 265s 3s/step - accuracy: 0.9969 - loss: 0.0485 - val_accuracy: 0.9979 - val_loss: 0.0201 - learning_rate: 5.0000e-06
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 0.9987 - loss: 0.0372 - val_accuracy: 0.9979 - val_loss: 0.0196 - learning_rate: 5.0000e-06


In [ ]:
# evaluation: load saved best models and evaluate on test set
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import matplotlib.pyplot as plt

# load
custom_best = load_model('custom_cnn_best.h5')  # from Cell 7
effnet_best = load_model('effnet_top_best.h5')  # from Cell 9/10

def get_truth_and_preds(model, dataset):
    y_true = []
    y_pred = []
    for x_batch, y_batch in dataset:
        probs = model.predict(x_batch, verbose=0)
        preds = np.argmax(probs, axis=1)
        y_true.append(y_batch.numpy())
        y_pred.append(preds)
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    return y_true, y_pred

if test_ds is not None:
    # expand test_ds to include all items if it used batching
    y_true, y_pred = get_truth_and_preds(effnet_best, test_ds)
    print("Classification report (EfficientNet):")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest')
    plt.title("Confusion matrix (EffNet)")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
else:
    print("No test set detected; use validation results for quick checks.")


No test set detected; use validation results for quick checks.


In [ ]:
# Model Comparison (Step 8 from Project Statement)

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Utility to get predictions from any model
def evaluate_model(model, dataset, model_name):
    y_true, y_pred = [], []
    for x_batch, y_batch in dataset:
        probs = model.predict(x_batch, verbose=0)
        preds = np.argmax(probs, axis=1)
        y_true.append(y_batch.numpy())
        y_pred.append(preds)
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    print(f"\nModel: {model_name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")

    return {
        "Model": model_name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-score": f1
    }










In [ ]:
# Compare both models
results = []
if test_ds is not None:
    results.append(evaluate_model(custom_best, test_ds, "Custom CNN"))
    results.append(evaluate_model(effnet_best, test_ds, "EfficientNetB0"))
else:
    results.append(evaluate_model(custom_best, val_ds, "Custom CNN (val)"))
    results.append(evaluate_model(effnet_best, val_ds, "EfficientNetB0 (val)"))


Model: Custom CNN (val)
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000

Model: EfficientNetB0 (val)
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000


In [ ]:
# Display results in a table
import pandas as pd
results_df = pd.DataFrame(results)
display(results_df)

In [ ]:
# Visual comparison chart
plt.figure(figsize=(8,5))
bar_width = 0.2
r1 = np.arange(len(results_df))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width*2 for x in r1]

plt.bar(r1, results_df['Accuracy'], width=bar_width, label='Accuracy')
plt.bar(r2, results_df['Precision'], width=bar_width, label='Precision')
plt.bar(r3, results_df['F1-score'], width=bar_width, label='F1-score')

plt.xticks([r + bar_width for r in range(len(results_df))], results_df['Model'])
plt.title("Model Comparison Metrics")
plt.xlabel("Model")
plt.ylabel("Score")
plt.legend()
plt.show()

In [ ]:
# Print which performed best
best_model_name = results_df.sort_values('F1-score', ascending=False).iloc[0]['Model']
print(f"🏆 Best Model Overall (by F1-score): {best_model_name}")

In [48]:
# save final models and class names
import json
effnet_best.save('final_effnet_model.keras')
custom_best.save('final_custom_cnn.keras')

with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

print("Saved: final_effnet_model.keras, final_custom_cnn.keras, class_names.json")
# Optionally copy results to Drive:
from google.colab import drive
# drive.mount('/content/drive')  # uncomment if not mounted
# shutil.copy('final_effnet_model.keras', '/content/drive/MyDrive/')

Saved: final_effnet_model.keras, final_custom_cnn.keras, class_names.json


In [52]:
#
app_code = r"""
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
import json

@st.cache_resource
def load_model_and_meta():
    import keras
    # ✅ Enable unsafe deserialization (we trust our own model file)
    keras.config.enable_unsafe_deserialization()

    # ✅ Load model with this flag
    model = tf.keras.models.load_model('final_effnet_model.keras', safe_mode=False)

    # ✅ Load class names
    with open('class_names.json', 'r') as f:
        class_names = json.load(f)
    return model, class_names


st.title("Brain Tumor MRI Classification")
st.write("Upload an MRI image and get a predicted tumor type with confidence scores.")

model, class_names = load_model_and_meta()

uploaded_file = st.file_uploader("Choose an MRI image", type=['png','jpg','jpeg'])
if uploaded_file is not None:
    img = Image.open(uploaded_file).convert('RGB')
    st.image(img, caption='Uploaded Image', use_column_width=True)
    st.write("Classifying...")
    img = img.resize((224,224))
    x = np.array(img).astype('float32') / 255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)[0]
    top_idx = preds.argmax()
    st.write(f"Predicted: **{class_names[top_idx]}** — confidence: **{preds[top_idx]*100:.2f}%**")

    st.write("All confidences:")
    for i, c in enumerate(preds):
        st.write(f"- {class_names[i]}: {c*100:.2f}%")
"""

with open("/content/app.py", "w") as f:
    f.write(app_code)
print("Wrote /content/app.py")

Wrote /content/app.py


In [32]:
from google.colab import files
uploaded = files.upload()

Saving isrgrootx1.pem to isrgrootx1.pem


In [53]:
from google.colab import files
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
!ls

app.py		    final_effnet_model.h5  sample_data
custom_cnn_best.h5  isrgrootx1.pem	   Tumour-20251109T042749Z-1-001.zip
data		    ngrok
effnet_top_best.h5  ngrok.zip


In [35]:
!pip install streamlit pyngrok mysql-connector-python pandas requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.7 MB/s eta 0:00:00


In [36]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip

Archive:  ngrok.zip
  inflating: ngrok                   


In [37]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "31UMSjeZERulE5B9MboU22TojhM_6Ffs5zbXgwSwrzyX1nSkw"

In [54]:
!streamlit run app.py --server.port 8501 &>/dev/null&

In [39]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🌍 Streamlit App URL:", public_url)

🌍 Streamlit App URL: NgrokTunnel: "https://613170ffd34e.ngrok-free.app" -> "http://localhost:8501"
